In [1]:
%pip install ortools
%pip install psutil

import os
import csv
import ast
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import psutil
from ortools.linear_solver import pywraplp

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import resource

def get_memory_usage_in_kb():
    process = psutil.Process()
    mem_info = process.memory_info()
    return mem_info.rss // 1024  # Convert bytes to KB

In [3]:
import resource

def set_memory_limit_gb(gb_limit):
    soft, hard = gb_limit * 1024 ** 3, gb_limit * 1024 ** 3  # Convert GB to bytes
    resource.setrlimit(resource.RLIMIT_AS, (soft, hard))

In [4]:
def parse_knapsack_csv(file_path):
    instances = []
    with open(file_path, 'r') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            weights = list(map(float, row['Weights'].strip('[]').split(',')))
            prices = list(map(float, row['Prices'].strip('[]').split(',')))
            capacity = float(row['knapsack_capacity'])
            items = [(i + 1, prices[i], weights[i]) for i in range(len(weights))]
            instances.append((len(items), items, weights, prices, capacity))
    return instances

In [ ]:
def solve_knapsack_with_GoogleOR(items, capacity, ram):
    set_memory_limit_gb(ram)
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        raise Exception("Solver not created.")

    num_items = len(items)
    x = [solver.BoolVar(f"x_{i}") for i in range(num_items)]
    
    profit = [item[1] for item in items]
    solver.Maximize(solver.Sum(profit[i] * x[i] for i in range(num_items)))
    
    weight = [item[2] for item in items]
    solver.Add(solver.Sum(weight[i] * x[i] for i in range(num_items)) <= capacity)
    solver.SetTimeLimit(300 * 1000) 

    result = {"objective_value": np.nan, "selected_items": [], "time_taken": np.nan,
              "iterations_count": None, "node_count": None, "peak_memory": np.nan}

    try:
        start_time = time.time()
        mem_before = get_memory_usage_in_kb()

        status = solver.Solve()

        mem_after = get_memory_usage_in_kb()
        end_time = time.time()

        result["time_taken"] = end_time - start_time
        result["peak_memory"] = max(mem_before, mem_after)

        if status == pywraplp.Solver.OPTIMAL:
            print("Optimal solution found")
            result["objective_value"] = solver.Objective().Value()
            result["selected_items"] = [items[i][0] for i in range(num_items) if x[i].solution_value() > 0.5]
        else:
            print("Solution not optimal/failed")

    except Exception as e:
        print(f"Error:: {e}")

    return result

In [ ]:
def process_knapsack_instances(input_csv_path, output_csv_path):
    # Parse the knapsack dataset
    instances = parse_knapsack_csv(input_csv_path)
    results = []
    ram = 256 #Specify the ram
    cpu_cores = 32 #Specify number_of_cpu_cores of your system
     
    for instance_idx, (n, items, weights, prices, capacity) in tqdm(enumerate(instances, start=1), total=len(instances), desc="Processing instances"):
        output = solve_knapsack_with_GoogleOR(items, capacity, ram)     
        result = {
            'number_of_elements': n,
            'capacity': capacity,
            'total_profit':  output["objective_value"],
            'solution_time': output["time_taken"],
            'ram': ram,
            'cpu_cores': cpu_cores,
            'peak_memory': output["peak_memory"]
        }
        results.append(result)

    
    fieldnames =  list(results[0].keys())
    
    with open(output_csv_path, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(merged_results)
    
    print(f"Results saved to {output_csv_path}")

In [ ]:
base_directory = "dataset_200.csv" #Path of input dataset file
output_csv = "output.csv" #Path of output file
process_knapsack_instances(base_directory, output_csv)

Processing instances:   1%|          | 2/199 [00:00<00:16, 12.01it/s]

Optimal solution found
Optimal solution found


Processing instances:   2%|▏         | 4/199 [00:00<00:20,  9.54it/s]

Optimal solution found
Optimal solution found


Processing instances:   3%|▎         | 5/199 [05:00<4:47:22, 88.88s/it]

...

Optimal solution found
Optimal solution found
Optimal solution found
Optimal solution found
Optimal solution found


Processing instances:  99%|█████████▉| 198/199 [23:40<00:01,  1.49s/it]

Optimal solution found


Processing instances: 100%|██████████| 199/199 [23:41<00:00,  7.14s/it]

Optimal solution found
Merged results saved to ./results_cpp/final_min/or/or_32_256.csv
